In [ ]:
# Task2: Proximity Sensors and Rule-Based Navigation
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import HTML, display

class WallEnvironment:
    """
    Environment with rectangular boundary and optional interior walls.
    `segments`: list of ((x1,y1),(x2,y2)) wall endpoints.
    """
    def __init__(self, size=100, walls=None):
        # Initializing square arena size and wall segments
        self.size = size
        b = size
        # Boundary walls form the square perimeter
        self.segments = [((0,0),(b,0)), ((b,0),(b,b)), ((b,b),(0,b)), ((0,b),(0,0))]
        # Adding interior walls
        if walls:
            self.segments.extend(walls)

    def intersect(self, origin, angle, max_range):
        """
        Casting a ray from `origin` at heading `angle`.
        Returning distance to nearest wall within `max_range`.
        Uses parametric line intersection on each segment.
        """
        ox, oy = origin
        dx, dy = np.cos(angle), np.sin(angle)
        min_t = max_range  # best (smallest) intersection distance
        for (x1,y1),(x2,y2) in self.segments:
            # Wall direction vector
            vx, vy = x2 - x1, y2 - y1
            # Solving for ray t and segment u using cross-product
            denom = dx * vy - dy * vx
            if abs(denom) < 1e-6:
                # Parallel: no intersection
                continue
            # Ray parameter t
            t = ((x1 - ox) * vy - (y1 - oy) * vx) / denom
            # Segment parameter u
            u = ((x1 - ox) * dy - (y1 - oy) * dx) / denom
            # Checking if intersection is within ray (0<=t<=max_range) and segment (0<=u<=1)
            if 0 <= t <= max_range and 0 <= u <= 1 and t < min_t:
                min_t = t
        return min_t

class ProximityRobot:
    """
    Robot with three forward-facing proximity sensors (left, front, right).
    Uses simple reactive rules to avoid obstacles.
    """
    def __init__(self, pos, phi, env, sensor_range=15.0, sensor_angle=np.pi/6):
        # `pos`: initial (x,y) position
        # `phi`: initial heading angle
        # `env`: WallEnvironment instance
        # `sensor_range`: max sensing distance
        # `sensor_angle`: side sensor offset
        self.pos = np.array(pos, float)
        self.phi = phi
        self.env = env
        self.range = sensor_range
        self.angle_offset = sensor_angle

    def read_proximity(self):
        """
        Measure normalized proximity in three directions:
        left (phi+offset), front (phi), right (phi-offset).
        Returns list [left_val, front_val, right_val], each in [0,1].
        """
        readings = []
        # test each sensor ray
        for offset in [self.angle_offset, 0, -self.angle_offset]:
            dist = self.env.intersect(self.pos, self.phi + offset, self.range)
            # convert distance to proximity value (1 near wall, 0 far)
            val = max(0.0, 1 - dist / self.range) if dist < self.range else 0.0
            readings.append(val)
        return readings

    def step(self, v=3.0, turn_rate=np.pi/4, dt=0.1):
        """
        Updating robot state using rule-based navigation:
        - If front sensor > threshold, turn away from closer side.
        - Else if side sensors > threshold, turn outward.
        - Move forward by speed v.
        Returns new position.
        """
        # reading sensor values
        left_val, front_val, right_val = self.read_proximity()
        # reactiving turning rules
        if front_val > 0.2:
            # obstacle ahead: turn away from side with stronger reading
            if left_val > right_val:
                self.phi -= turn_rate * dt
            else:
                self.phi += turn_rate * dt
        elif left_val > 0.2:
            # obstacle on left: turn right
            self.phi += turn_rate * dt
        elif right_val > 0.2:
            # obstacle on right: turn left
            self.phi -= turn_rate * dt
        # moving forward in heading direction
        dx = v * dt * np.cos(self.phi)
        dy = v * dt * np.sin(self.phi)
        self.pos += np.array([dx, dy])
        # confine to environment bounds
        self.pos = np.clip(self.pos, 0, self.env.size)
        return self.pos.copy()

# --- Setup simulation ---
walls = [((30,30),(70,30)), ((70,30),(70,70)), ((70,70),(30,70))]
env = WallEnvironment(size=100, walls=walls)
robot = ProximityRobot(pos=(10,10), phi=0.0, env=env)
steps, dt = 200, 0.1
traj = np.zeros((steps,2))
for i in range(steps):
    traj[i] = robot.pos.copy()  # record trajectory
    robot.step(dt=dt)            # advance robot

# --- Static plot ---
plt.figure(figsize=(6,6))
# drawing walls
for (x1,y1),(x2,y2) in env.segments:
    plt.plot([x1,x2], [y1,y2], 'k-')
# drawing robot path
plt.plot(traj[:,0], traj[:,1], 'r-', label='Proximity Robot')
plt.title('Proximity Navigation Trajectory')
plt.xlabel('X'); plt.ylabel('Y')
plt.legend(loc='upper right', fontsize='small')
plt.savefig('task2_traj.png', bbox_inches='tight'); display(plt.gcf()); plt.close()

# --- Animation ---
fig2, ax2 = plt.subplots(figsize=(6,6))
ax2.set_xlim(0,100); ax2.set_ylim(0,100)
# initial wall drawing
for (x1,y1),(x2,y2) in env.segments:
    ax2.plot([x1,x2], [y1,y2], 'k-')
# initializing line for robot
line, = ax2.plot([], [], 'r-', lw=2, label='Proximity Robot')
ax2.legend(loc='upper right')

def init2():
    """Clear robot line before animation."""
    line.set_data([], [])
    return line,

def update2(frame):
    """Update robot line up to current frame."""
    line.set_data(traj[:frame,0], traj[:frame,1])
    return line,

anim2 = FuncAnimation(fig2, update2, frames=steps, init_func=init2, blit=True)
anim2.save('task2_anim.gif', writer=PillowWriter(fps=20))
plt.close()
display(HTML('<img src="task2_anim.gif" />'))
